**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов

In [32]:
import re
import gzip
from random import sample
from itertools import chain
from pprint import pprint
from operator import mul
from functools import reduce 
from collections import Counter
from collections import OrderedDict
# from collections import defaultdict

#import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

In [2]:
# !ls ../data

In [3]:
file_name = '../data/dostoevsky-besy-p2.txt.gz'

with gzip.open(file_name,'rt') as f:  text = f.read()[105:] # выкидываем заголовок
    
print('символов:%i\n'%(len(text)))
print(text[:364].strip())

символов:465490

Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.


In [4]:
text_ = [ 
    word_tokenize(
        re.sub(r'[^а-яa-z-]', ' ', s.lower()) # выкидываем лишние символы
        # s.lower()
    ) # разбиваем предложения на слова
    for s in sent_tokenize(text) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(text_)))

sample(text_,2)

предложений: 5556



[['-',
  'уговаривал',
  'николай',
  'всеволодович',
  'но',
  'на',
  'этот',
  'раз',
  'долго',
  'не',
  'мог',
  'уговорить',
  'она',
  'молча',
  'смотрела',
  'на',
  'него',
  'все',
  'с',
  'тем',
  'же',
  'мучительным',
  'недоумением',
  'с',
  'тяжелою',
  'мыслию',
  'в',
  'своей',
  'бедной',
  'голове',
  'и',
  'все',
  'так',
  'же',
  'усиливаясь',
  'до',
  'чего-то',
  'додуматься'],
 ['-', 'прошу', 'слова', '-', 'угрюмо', 'но', 'твердо', 'заявил', 'шигалев']]

----

In [5]:
text_tokens = list(chain(*text_)) # собираем все токены (слова) из текста
vocab_size = len(set(text_tokens))

text_tokens_freq = Counter( text_tokens ) # оценка частоты использования слов

print('всего слов тексте: %i'%(len(text_tokens)))
print('размер словаря: %i'%(vocab_size))
text_tokens_freq.most_common()[:10] # наиболее частые токены

всего слов тексте: 74977
размер словаря: 15670


[('-', 3005),
 ('и', 2983),
 ('не', 1777),
 ('в', 1750),
 ('что', 1310),
 ('я', 1267),
 ('на', 891),
 ('с', 862),
 ('он', 822),
 ('вы', 772)]

----

In [6]:
# вынимаем все n-gram из текста
ngram_len = 2 # работаем с биграммами
text_ngrams = [ ngram for s in text_ for ngram in ngrams(s,ngram_len) ]
print('количество n-gram: %i'%(len(set(text_ngrams))))
sample(text_ngrams,5)

количество n-gram: 49001


[('время', 'своими'),
 ('личность', 'во'),
 ('наконец', 'спросить'),
 ('провидению', 'руками'),
 ('не', 'сыскала')]

In [7]:
# cчитаем частоту n-gram
text_ngrams_freq = Counter(text_ngrams)
sample( text_ngrams_freq.items(), 7)

[(('называется', 'merci'), 1),
 (('пьяны', 'церкви'), 1),
 (('по', 'почте'), 4),
 (('было', 'стать'), 1),
 (('еще', 'при'), 2),
 (('низводите', 'до'), 1),
 (('все', 'но'), 1)]

In [8]:
text_ngrams_freq.most_common()[:10] # наиболее частые n-ngram

[(('-', 'я'), 201),
 (('николай', 'всеволодович'), 166),
 (('и', 'не'), 156),
 (('петр', 'степанович'), 151),
 (('я', 'не'), 111),
 (('что', 'я'), 102),
 (('потому', 'что'), 101),
 (('что', 'вы'), 98),
 (('-', 'вы'), 93),
 (('как', 'бы'), 89)]

----

Оценка вероятностей совместного использования слов со сглаживанием Лапласа


$$
P(w_n|w_{n-1}) = \frac{ C(w_{n-1} w_{n}) +1 }{ C(w_{n-1}) + V }
$$

In [9]:
text_ngrams_prob = { # оценка совместного использования токенов
    ngram : (text_ngrams_freq[ngram]+1) / ( text_tokens_freq[ ngram[0] ] + vocab_size )
    for ngram in text_ngrams_freq 
}

text_ngrams_prob

{('теперь', 'когда'): 0.0003785488958990536,
 ('когда', 'уже'): 0.0004431501645986326,
 ('уже', 'все'): 0.0002523181732164259,
 ('все', 'прошло'): 0.00012366289494837075,
 ('прошло', 'и'): 0.00019141198239009763,
 ('и', 'я'): 0.0018763737736557122,
 ('я', 'пишу'): 0.00017712700005904233,
 ('пишу', 'хронику'): 0.00012761613067891782,
 ('хронику', 'мы'): 0.00012761613067891782,
 ('мы', 'уже'): 0.00012657426745142712,
 ('уже', 'знаем'): 0.00012615908660821296,
 ('знаем', 'в'): 0.00012758356723653993,
 ('в', 'чем'): 0.0010907003444316878,
 ('чем', 'дело'): 0.00031721862707778203,
 ('дело', 'но'): 0.0001905608842025027,
 ('но', 'тогда'): 0.00012242899118511264,
 ('тогда', 'мы'): 0.00012685525815045033,
 ('мы', 'еще'): 0.0003164356686285678,
 ('еще', 'ничего'): 0.00025081514923501377,
 ('ничего', 'не'): 0.0036075949367088606,
 ('не', 'знали'): 0.00034389866452685274,
 ('знали', 'и'): 0.00012751849018107625,
 ('и', 'естественно'): 0.00010722135849461212,
 ('естественно', 'что'): 0.00012762427

---

по предыдущим словам подобираем слово с наибольшей вероятностью 

$$
P(w_1,\ldots, w_n) = \prod_{k=1}^{n} P(w_k|w_k-1)
$$

In [31]:
def get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    if ngram in text_ngrams_prob: return text_ngrams_prob[ngram]
    token_0_freq = text_tokens_freq[ngram[0]] if ngram[0] in text_tokens_freq else 0.
    return 1./(token_0_freq+len(text_tokens_freq))

In [27]:
def get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    if ngram in text_ngrams_prob: return text_ngrams_prob[ngram]
    token_0_freq = text_tokens_freq[ngram[0]] if ngram[0] in text_tokens_freq else 0.
    return 1./(token_0_freq+len(text_tokens_freq))

In [28]:
# оценка предложения
def get_sentence_prob(sentence,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    return reduce( mul, [ 
        get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in ngrams(sentence,ngram_len) 
    ] )

# get_sentence_prob(sentence)

In [30]:
# оценка всех возможных продолжений предложения
def get_next_token_prob(sentence,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    
    sentence_prob = get_sentence_prob( # оценка предложения
            sentence,
            text_ngrams_prob=text_ngrams_prob,
            text_tokens_freq=text_tokens_freq
        )
    
    token_next = { # оценки всех возможных продолжений
        ngram[1] : 
            sentence_prob*
              get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in text_ngrams_prob if ngram[0]==sentence[-1] 
    }
    
    return token_next

# get_next_token_prob(sentence)

In [36]:
# from collections import defaultdict

In [66]:
def get_top_prob_token(tokens_prob,n=3):  # n наиболее вероятных продолжений
    kf = lambda k: tokens_prob[k]
    return OrderedDict([
        (key,tokens_prob[key])
        for key in sorted(tokens_prob, key=kf, reverse=True)[:n]
    ])

# get_top_prob_token( get_next_token_prob(sentence) )

----

In [70]:
# генерируем продолжения

for sentence in sample(text_,10):
    if len(sentence)<5: continue
    tokens_next = get_top_prob_token( get_next_token_prob(sentence[:-2]) )
        
    print( 
        ' '.join(sentence[:-2])+  '...' + str(list(tokens_next.keys())) + '\n'
    )

- согласен согласен я с вами совершенно согласен но это у нас рано рано -...['я', 'вы', 'да']

- зато тебе выгодно как шпиону...['и']

в этом интимном кружке толпившемся около нее конечно между молодежью позволялось и даже вошло в правило делать разные шалости - действительно иногда...['впрочем', 'тоже', 'рассмешу']

- и в новый суд его не потащил-с -...['я', 'вы', 'да']

казалось это был сильный брюнет сухощавый и смуглый глаза были большие непременно черные с сильным блеском и с желтым отливом как у цыган это и в...['этом', 'то', 'самом']

да хоть бы с третьего а не со второго слова а...['не', 'вы', 'я']

вы ко мне пристаете почти что...['я', 'вы', 'он']

вам ничего не значит пожертвовать жизнью и своею...['рукой', 'истиной', 'дорогой']

чрезмерная подписка манила на расходы хотели сделать что-то чудесное - вот почему...['же', 'я', 'мне']

